In [2]:
import numpy as np
import pennylane as qml

![circuit](./images/H.4.1.1.png)

**Codercise H.4.1.** Implement the circuit drawn above, allowing for the specification of an initial computational basis state.

Tip. Try using [qml.BasisState](https://docs.pennylane.ai/en/stable/code/api/pennylane.BasisState.html) to prepare the basis state.

In [3]:
n_bits = 2
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def zz_circuit(alpha, time, init):
    """Circuit for evolving two electrons with a ZZ interaction.

    Args:
        alpha (float): The strength of the interaction.
        time (float): The time we evolve the electron wavefunction for.
        init (numpy.array(int)): An initial state specified by two bits [x, y]. Prepare the
            system in this state prior to applying the time evolution circuit.

    Returns:
        array[float]: Probabilities for observing different outcomes.
    """
    hbar = 1e-34

    qml.BasisState(init, wires=range(n_bits))
    qml.CNOT([0,1])
    qml.RZ(2*alpha*time/hbar, wires=1)
    qml.CNOT([0,1])

    return qml.probs(wires=range(n_bits))

Physically speaking, this setup naturally arises if we have a lattice of electrons which interact only with each other and not with a background magnetic field. Sometimes, the interaction means they want to anti-align, i.e., the energy will be lowered when adjacent electrons have spins in opposite directions. Thus, the graph problem is a natural generalization of a single tiny bar magnet!

**Codercise H.4.2.**
(a) Complete the following code to construct the Hamiltonian. The remaining code defines a circuit `energy(init)` which gives the energy expectation in a basis state `init`.

In [5]:
n_bits = 5
dev = qml.device("default.qubit", wires=n_bits)

obs = [qml.PauliZ(0) @ qml.PauliZ(1), qml.PauliZ(1) @ qml.PauliZ(2), qml.PauliZ(1)@qml.PauliZ(3), qml.PauliZ(3) @ qml.PauliZ(4)]
coeffs = [1 for _ in obs]
H = qml.Hamiltonian(coeffs, obs)


@qml.qnode(dev)
def energy(init):
    """Circuit for measuring expectation value of Hamiltonian in a given state.

    Args:
        init (numpy.array(int)): An initial computational basis state, specified by five bits.

    Returns:
        float: Expectation value of the Hamiltonian H.
    """
    qml.BasisState(init, wires=range(n_bits))
    return qml.expval(H)

The ground state minimizes the energy. If the graph is bipartite, then we can "colour" the nodes with states $\Ket{0}$ and $\Ket{1}$ so that any pair has distinct states on either end and the edge contributes $-1$ to the energy. Thus, the ground state energy will be $-4$ just in case the graph is bipartite, and higher otherwise.

(b) Determine the two ground states and check they give the expected energy.

In [9]:
my_guess1 = np.array([0,1,0,0,1])
my_guess2 = np.array([1,0,1,1,0])

print("The expected energy for", my_guess1, "is", energy(my_guess1), ".")
print("The expected energy for", my_guess2, "is", energy(my_guess2), ".")

The expected energy for [0 1 0 0 1] is -4.0 .
The expected energy for [1 0 1 1 0] is -4.0 .
